## Вывод информации о данных в табличном виде

In [ ]:
def table_feature_analysis(*, data: pd.DataFrame, cat_features: list = [], num_features: list = []) -> object:
    ''' 
    Функция для вывода таблицы с общей информацией о категориальных и числовых признаках в датафрейма.
    
    На вход функция принимает:
    1. data - датафрейм с данными;
    2. cat_features - список с названиями столбцов, содержащих категориальные данные;
    3. num_features - список с названиями столбцов числовых признаков.
    
    На выходе функция выводит таблицу со следующими данными по всем признакам:
    1. Тип данных в исходном датафрейме для каждого признака.
    2. Количество данных в исходном датафрейме для каждого признака.
    3. Количество пропусков в исходном датафрейме для каждого признака.
    4. Доля пропусков от общего количества строк в датафрейме для каждого признака.
    
    Дополнительно для числовых признаков в итоговой таблице расчитываются статистические данные о признаке.    
    '''
    import pandas as pd
    
    all_features = num_features + cat_features
    
    binar_cat_features = []
    for feature in cat_features:
        if data[feature].nunique() < 3:
            binar_cat_features.append(feature)
    
    data = data[all_features]
    
    desc = pd.DataFrame(index=data.columns.to_list())
    
    desc['type'] = df.dtypes
    desc['count'] = df.count()
    desc['null'] = df.isna().sum()
    desc['%null'] = desc['null'] / len(df) * 100
           
    if num_features or binar_cat_features:
        desc = pd.concat([desc, data[num_features + binar_cat_features].describe().T.drop('count', axis=1)], axis=1)
        desc.loc[num_features + binar_cat_features, "skew"] = data[num_features].skew()
        desc.loc[num_features + binar_cat_features, "kurtosis"] = data[num_features].kurtosis()
    
    if cat_features:
        desc['nunique'] = df[cat_features].nunique()
        
    desc = desc.round(2)
    
    return desc

## Вывод информации о данных в графическом виде

In [ ]:
def graph_feature_analysis(*, data: pd.DataFrame, cat_features: list = [], num_features: list = [], target: str):
    ''' 
    Функция для вывода графичиской информации о категориальных и числовых признаках 
    с разделением по целевому признаку (для задачи классификации).
    
    На вход функция принимает:
    1. data - датафрейм с данными;
    2. cat_features - список с названиями столбцов категориальных признаков;
    3. num_features - список с названиями столбцов числовых признаков;
    4. target - наименование колонки с целевым признаком.
    
    На выходе функция выводит следующие графикии:
    1. Для числовых данных - гистограммы, разделённые по целевому признаку.
    2. Для категориальных данных - столбчатые диаграммы, разделённые по целевому признаку.
    '''

    import pandas as pd
    import matplotlib.pyplot as plt
    import seaborn as sns
    
    if target in cat_features:
        cat_features.remove(target)
    elif target in num_features:
        num_features.remove(target)
    
    all_features = num_features + cat_features
    
    data = data[all_features + [target]]
    
    num_of_lines = len(all_features)
    
    plt.figure(figsize = (13, int(num_of_lines*2)))
    
    for i, col in enumerate(all_features):
        plt.subplot(num_of_lines // 2 + num_of_lines % 2, 2, i+1)
        plt.grid()
        if col in num_features:
            sns.histplot(data=data, x=col, kde=True, hue=target,  palette='tab10', fill=True, bins = 80)
        else:
            sns.countplot(data=data, x=col, hue=target)
            
    plt.tight_layout()
    plt.show()

## Оценка наличия зависимостей параметров друг от друга в графическом виде

In [ ]:
def dependency_analysis(*, data: pd.DataFrame, columns: list = []):
    '''
    Функция для оценки наличия зависимостей параметров между собой в графическом виде.
    
    На вход функция принимает:
    1. data - датафрейм с данными;
    2. columns - список колонок, которые необходимо оценить.
    
    На выходе функция выводит следующие графикии:
    1. Гистограмму данных по каждой колонке из списка columns.
    2. Графики зависимостей всех параметров друг от друга.
    
    '''
    
    import pandas as pd
    import matplotlib.pyplot as plt
    
    for i in range(len(columns)):
        
        print('*'*30, 'Колонка ', cols_list[i], '*'*30)
        fig, ax = plt.subplots(figsize=(5, 3), layout='constrained')
        ax.set_xlabel(columns[i])
        ax.set_ylabel('Количество данных')
        ax.set_title("Гистограмма")
        plt.hist(df[cols_list[i]], bins=30)
        plt.grid()
        plt.show()
        
        num_graphs = len(cols_list) - i - 1
        
        if num_graphs > 2:
            fig, axs = plt.subplots(num_graphs//2+num_graphs%2, 2, figsize=(13, int(num_graphs*3)))
        
            for k in range(i+1, len(cols_list)):
                idx_graph = k-i-1
                print(i, k, idx_graph, idx_graph//2, idx_graph%2)
                axs[idx_graph//2, idx_graph%2].scatter(df[cols_list[k]], df[cols_list[i]])
                axs[idx_graph//2, idx_graph%2].set_xlabel(cols_list[k])  # Add an x-label to the Axes.
                axs[idx_graph//2, idx_graph%2].set_ylabel(cols_list[i])  # Add a y-label to the Axes.
                axs[idx_graph//2, idx_graph%2].set_title('Зависимость '+cols_list[i]+' от '+ str(cols_list[k]))
                axs[idx_graph//2, idx_graph%2].grid()
            
            plt.show()
        
        elif num_graphs == 2 or num_graphs == 1:
            fig, axs = plt.subplots(1, 2, figsize=(13, int(num_graphs*3)))
        
            for k in range(i+1, len(cols_list)):
                idx_graph = k-i-1
                print(i, k, idx_graph, idx_graph//2, idx_graph%2)
                axs[idx_graph%2].scatter(df[cols_list[k]], df[cols_list[i]])
                axs[idx_graph%2].set_xlabel(cols_list[k])  # Add an x-label to the Axes.
                axs[idx_graph%2].set_ylabel(cols_list[i])  # Add a y-label to the Axes.
                axs[idx_graph%2].set_title('Зависимость '+cols_list[i]+' от '+ str(cols_list[k]))
                axs[idx_graph%2].grid()
            
            plt.show()